# The Battle of the Neighborhoods - Week 1
## Karen's Capstone Project - Waste and recycling top neighborhoods in Porto District - Portugal

### Waste and recycling top neighborhoods in Porto District - Portugal

#### 1. A description of the problem and a discussion of the background. (15 marks)

#### Introduction

Sustainability has became a big word in the past 30 years. Neglected by years not to sacrifice the so-called development by most companies and ignored by the average citizen, being green is finally becoming natural and, why not?, cool. Social responsability leads the game for outstanding in competitive markets as new crowds look for reducing its environmental impact. 

But how are portuguese people dealing with their own waste at home?

To understand what happpens behind closed doors, this project will display how homemade waste is treated across the District of Porto. 

I believe this can be the key to understanding good policies to expand across communities and where, and maybe what, is missing in the areas where waste is not treated well by its families.


#### Target audience
Waste treatment companies, portuguese public offices, portuguese community

#### Goals 
The goal of this report is to go understand the differences in between waste treatment in Porto District by families and then propose new action plans to expand awareness.


#### 2. A description of the data and how it will be used to solve the problem. (15 marks)

#### Data required
a) Neighborhoods 
Retrieved from https://raw.githubusercontent.com/centraldedados/codigos_postais/master/data/codigos_postais.csv

b) Districts
Retrieved from https://raw.githubusercontent.com/centraldedados/codigos_postais/master/data/distritos.csv

c) Concelhos
Retrieved from https://raw.githubusercontent.com/centraldedados/codigos_postais/master/data/concelhos.csv

d) Waste data
Retrieved from several data sets available on Dados.gov
https://dados.gov.pt/en/

e) Foursquare API
To visualize results  

#### Retrieving sample of data to be used

In [1]:
#need to instal
!conda install -c anaconda beautifulsoup4 --yes
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge json-flatten --yes
!conda install -c conda-forge json_normalize --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |           py36_0         168 KB  anaconda
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    certifi-2020.4.5.1         |           py36_0         159 KB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    soupsieve-2.0.1            |             py_0          33 KB  anaconda
    ------------------------------------------------------------
                                           Total:         4.3 MB

The following NEW packages will be INSTALLED:

  beautifulsoup4     anaconda/linux-64::beautifulsoup4-4.9.1-py36_0
  soupsieve          anaconda/noarch::soupsieve-2.0.1-py_0

The following packag

In [2]:
#need to import
import numpy as np 
import pandas as pd 
import requests 
import json  
import io
from pandas import json_normalize
#import folium 
#from geopy.geocoders import Nominatim


print('Import completed. Get started!')

Import completed. Get started!


In [3]:
!wget -O zipcode.csv https://raw.githubusercontent.com/centraldedados/codigos_postais/master/data/codigos_postais.csv
postalcode = pd.read_csv('zipcode.csv')
postalcode.head()

--2020-06-15 19:18:58--  https://raw.githubusercontent.com/centraldedados/codigos_postais/master/data/codigos_postais.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.124.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.124.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26823908 (26M) [text/plain]
Saving to: ‘zipcode.csv’

zipcode.csv         100%[===================>]  25.58M  23.2MB/s    in 1.1s    

2020-06-15 19:19:00 (23.2 MB/s) - ‘zipcode.csv’ saved [26823908/26823908]



,cod_distrito,cod_concelho,cod_localidade,nome_localidade,cod_arteria,tipo_arteria,prep1,titulo_arteria,prep2,nome_arteria,local_arteria,troco,porta,cliente,num_cod_postal,ext_cod_postal,desig_postal
0,1,1,249,Alcafaz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3750,11,AGADÃO
1,1,1,250,Caselho,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3750,12,AGADÃO
2,1,1,251,Corga da Serra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3750,13,AGADÃO
3,1,1,252,Foz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3750,14,AGADÃO
4,1,1,253,Guistola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3750,15,AGADÃO


In [4]:
!wget -O distritos.csv https://raw.githubusercontent.com/centraldedados/codigos_postais/master/data/distritos.csv
distrito = pd.read_csv('distritos.csv')
distrito.head()

--2020-06-15 19:19:02--  https://raw.githubusercontent.com/centraldedados/codigos_postais/master/data/distritos.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.124.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.124.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 450 [text/plain]
Saving to: ‘distritos.csv’

distritos.csv       100%[===================>]     450  --.-KB/s    in 0s      

2020-06-15 19:19:03 (18.3 MB/s) - ‘distritos.csv’ saved [450/450]



,cod_distrito,nome_distrito
0,1,Aveiro
1,2,Beja
2,3,Braga
3,4,Bragança
4,5,Castelo Branco


In [5]:
!wget -O concelhos.csv https://raw.githubusercontent.com/centraldedados/codigos_postais/master/data/concelhos.csv
concelhos = pd.read_csv('concelhos.csv')
concelhos.head()

--2020-06-15 19:19:03--  https://raw.githubusercontent.com/centraldedados/codigos_postais/master/data/concelhos.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.124.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.124.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5514 (5.4K) [text/plain]
Saving to: ‘concelhos.csv’

concelhos.csv       100%[===================>]   5.38K  --.-KB/s    in 0.002s  

2020-06-15 19:19:03 (3.44 MB/s) - ‘concelhos.csv’ saved [5514/5514]



,cod_distrito,cod_concelho,nome_concelho
0,1,6,Castelo de Paiva
1,1,7,Espinho
2,1,8,Estarreja
3,1,9,Santa Maria da Feira
4,1,10,Ílhavo


In [ ]:
#my foursquare data
CLIENT_ID = 'NRVYHZ12WQEQIP4HSL5ATU4NH45OWYF5MGX2B0CIKK4OVJAK' 
CLIENT_SECRET = 'HOA3LBJ33NHG5EK0HYV2V0JGTQMMUBCEN4XG45MTL3NEK5X1' 
VERSION = '20180604'
LIMIT = 200

In [6]:
!wget -O lixo.json https://dados.gov.pt/pt/datasets/r/459ea19b-a966-4c24-a46b-2e1572e13aa5 --no-check-certificate

with open('lixo.json') as f: 
    d = json.load(f) 
#print(d)
lixo = json_normalize(d['d']) 
lixo.head() 

--2020-06-15 19:19:03--  https://dados.gov.pt/pt/datasets/r/459ea19b-a966-4c24-a46b-2e1572e13aa5
Resolving dados.gov.pt (dados.gov.pt)... 62.28.186.239
Connecting to dados.gov.pt (dados.gov.pt)|62.28.186.239|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://dados.gov.pt/s/dadosGovFiles/INE0000480.json [following]
--2020-06-15 19:19:04--  https://dados.gov.pt/s/dadosGovFiles/INE0000480.json
Reusing existing connection to dados.gov.pt:443.
HTTP request sent, awaiting response... 200 OK
Length: 248509 (243K) [application/json]
Saving to: ‘lixo.json’

lixo.json           100%[===================>] 242.68K   693KB/s    in 0.4s    

2020-06-15 19:19:04 (693 KB/s) - ‘lixo.json’ saved [248509/248509]



,PartitionKey,RowKey,Timestamp,entityid,cod1,periododereferenciadosdados,cod2,valor
0,INE0000480,635976517670963620,2016-04-30T22:22:48.006148Z,f6919dc5-9e68-41b5-8965-5fbefe874293,S7A2011,2011,PT,490
1,INE0000480,635976517677682671,2016-04-30T22:22:48.7482409Z,8151bf27-3697-4b46-9dc5-6da51ff56e6f,S7A2013,2013,PT,440
2,INE0000480,635976517684714236,2016-04-30T22:22:49.3813227Z,b361da41-ec10-4db0-a2d2-8f4a609d08ce,S7A2012,2012,PT,453
3,INE0000480,635976517691589544,2016-04-30T22:22:50.06441Z,b6224ebc-654c-4e2a-8d83-4d0290337e4e,S7A2012,2012,1,452
4,INE0000480,635976517698777366,2016-04-30T22:22:50.770499Z,e2760a28-7eac-4a44-b67f-d587256cf90b,S7A2013,2013,1,439


In [7]:
!wget -O residuos_urbanos_selectiva.json https://dados.gov.pt/pt/datasets/r/44bf80c0-3c5f-4c8f-b27c-9439f0a8b3f3 --no-check-certificate

with open('residuos_urbanos_selectiva.json') as h: 
    g = json.load(h) 

residuos_urbanos_selectiva = json_normalize(g[0]['Dados']['2014']) 
residuos_urbanos_selectiva.head()

--2020-06-15 19:19:05--  https://dados.gov.pt/pt/datasets/r/44bf80c0-3c5f-4c8f-b27c-9439f0a8b3f3
Resolving dados.gov.pt (dados.gov.pt)... 62.28.186.239
Connecting to dados.gov.pt (dados.gov.pt)|62.28.186.239|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd=0008661&lang=PT [following]
--2020-06-15 19:19:05--  https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd=0008661&lang=PT
Resolving www.ine.pt (www.ine.pt)... 172.67.41.182, 104.22.20.250, 104.22.21.250, ...
Connecting to www.ine.pt (www.ine.pt)|172.67.41.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘residuos_urbanos_selectiva.json’

residuos_urbanos_se     [    <=>             ] 437.97K   624KB/s    in 0.7s    

2020-06-15 19:19:07 (624 KB/s) - ‘residuos_urbanos_selectiva.json’ saved [448485]



,geocod,geodsg,dim_3,dim_3_t,sinal_conv,sinal_conv_desc,valor
0,1111605,Paredes de Coura,T,Total,&,Dado provisório,233.196
1,1120310,Terras de Bouro,T,Total,&,Dado provisório,267.149
2,1190307,Fafe,T,Total,&,Dado provisório,1622.377
3,11A1315,Valongo,T,Total,&,Dado provisório,4866.935
4,11C1305,Lousada,T,Total,&,Dado provisório,867.207


In [8]:
!wget -O residuos_recolhidos.json https://dados.gov.pt/pt/datasets/r/d468d6df-0d85-4238-b7f4-e41e862b347e --no-check-certificate

with open('residuos_recolhidos.json') as j: 
    k = json.load(j) 

coleta = json_normalize(k[0]['Dados']['2018']) 
coleta.head() 

--2020-06-15 19:19:07--  https://dados.gov.pt/pt/datasets/r/d468d6df-0d85-4238-b7f4-e41e862b347e
Resolving dados.gov.pt (dados.gov.pt)... 62.28.186.239
Connecting to dados.gov.pt (dados.gov.pt)|62.28.186.239|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd=0009612&lang=PT [following]
--2020-06-15 19:19:08--  https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd=0009612&lang=PT
Resolving www.ine.pt (www.ine.pt)... 104.22.20.250, 104.22.21.250, 172.67.41.182, ...
Connecting to www.ine.pt (www.ine.pt)|104.22.20.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘residuos_recolhidos.json’

residuos_recolhidos     [  <=>               ] 156.29K   438KB/s    in 0.4s    

2020-06-15 19:19:09 (438 KB/s) - ‘residuos_recolhidos.json’ saved [160041]



,geocod,geodsg,dim_3,dim_3_t,valor,sinal_conv,sinal_conv_desc
0,11A1315,Valongo,1,Recolha indiferenciada,32968,NaN,NaN
1,11C1305,Lousada,1,Recolha indiferenciada,17653,NaN,NaN
2,11C0106,Castelo de Paiva,1,Recolha indiferenciada,5134,NaN,NaN
3,11A0116,São João da Madeira,1,Recolha indiferenciada,9697,NaN,NaN
4,11D1818,Sernancelhe,1,Recolha indiferenciada,1587,NaN,NaN


In [9]:
!wget -O residuos_urbana_por_habitante.json https://dados.gov.pt/pt/datasets/r/60d193bc-b80f-4cd4-8e16-37152f46aa65 --no-check-certificate

with open('residuos_urbana_por_habitante.json') as l: 
    m = json.load(l) 
    
residuos_urbana_por_habitante = json_normalize(m[0]['Dados']['2018']) 
residuos_urbana_por_habitante.head() 

--2020-06-15 19:19:09--  https://dados.gov.pt/pt/datasets/r/60d193bc-b80f-4cd4-8e16-37152f46aa65
Resolving dados.gov.pt (dados.gov.pt)... 62.28.186.239
Connecting to dados.gov.pt (dados.gov.pt)|62.28.186.239|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd=0008657&lang=PT [following]
--2020-06-15 19:19:10--  https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd=0008657&lang=PT
Resolving www.ine.pt (www.ine.pt)... 104.22.21.250, 104.22.20.250, 172.67.41.182, ...
Connecting to www.ine.pt (www.ine.pt)|104.22.21.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘residuos_urbana_por_habitante.json’

residuos_urbana_por     [ <=>                ]  32.62K  --.-KB/s    in 0.007s  

2020-06-15 19:19:11 (4.41 MB/s) - ‘residuos_urbana_por_habitante.json’ saved [33401]



,geocod,geodsg,valor,sinal_conv,sinal_conv_desc
0,16I1421,Ourém,378,NaN,NaN
1,16I1418,Tomar,465,NaN,NaN
2,16I1410,Entroncamento,413,NaN,NaN
3,1701511,Sesimbra,673,NaN,NaN
4,1810211,Odemira,593,NaN,NaN


In [10]:
!wget -O recolha_selectiva_por_habitante.json https://dados.gov.pt/pt/datasets/r/bbde6a1b-888a-4639-8c08-37f4f26aae71 --no-check-certificate

with open('recolha_selectiva_por_habitante.json') as n: 
    o = json.load(n) 
   
recolha_selectiva_por_habitante = json_normalize(o[0]['Dados']['2018']) 
recolha_selectiva_por_habitante.head() 

--2020-06-15 19:19:11--  https://dados.gov.pt/pt/datasets/r/bbde6a1b-888a-4639-8c08-37f4f26aae71
Resolving dados.gov.pt (dados.gov.pt)... 62.28.186.239
Connecting to dados.gov.pt (dados.gov.pt)|62.28.186.239|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd=0008660&lang=PT [following]
--2020-06-15 19:19:12--  https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd=0008660&lang=PT
Resolving www.ine.pt (www.ine.pt)... 172.67.41.182, 104.22.21.250, 104.22.20.250, ...
Connecting to www.ine.pt (www.ine.pt)|172.67.41.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘recolha_selectiva_por_habitante.json’

recolha_selectiva_p     [ <=>                ]  32.38K  --.-KB/s    in 0.1s    

2020-06-15 19:19:13 (327 KB/s) - ‘recolha_selectiva_por_habitante.json’ saved [33152]



,geocod,geodsg,valor,sinal_conv,sinal_conv_desc
0,16H0506,Oleiros,51,NaN,NaN
1,16J0503,Covilhã,50,NaN,NaN
2,16J0501,Belmonte,41,NaN,NaN
3,16B1108,Lourinhã,109,NaN,NaN
4,1701107,Loures,50,NaN,NaN


In [11]:
!wget -O proporcao_total_recicla.json https://dados.gov.pt/pt/datasets/r/0a8cba6f-9a82-4e11-9794-9e872ede8cc7 --no-check-certificate

with open('proporcao_total_recicla.json') as p: 
    q = json.load(p) 
   
proporcao_total_recicla = json_normalize(q[0]['Dados']['2018']) 
proporcao_total_recicla.head() 

--2020-06-15 19:19:13--  https://dados.gov.pt/pt/datasets/r/0a8cba6f-9a82-4e11-9794-9e872ede8cc7
Resolving dados.gov.pt (dados.gov.pt)... 62.28.186.239
Connecting to dados.gov.pt (dados.gov.pt)|62.28.186.239|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd=0009199&lang=PT [following]
--2020-06-15 19:19:14--  https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd=0009199&lang=PT
Resolving www.ine.pt (www.ine.pt)... 172.67.41.182, 104.22.20.250, 104.22.21.250, ...
Connecting to www.ine.pt (www.ine.pt)|172.67.41.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘proporcao_total_recicla.json’

proporcao_total_rec     [ <=>                ]  32.43K  --.-KB/s    in 0.1s    

2020-06-15 19:19:15 (280 KB/s) - ‘proporcao_total_recicla.json’ saved [33206]



,geocod,geodsg,valor,sinal_conv,sinal_conv_desc
0,1701508,Palmela,28,NaN,NaN
1,1701115,Amadora,35.5,NaN,NaN
2,16I1408,Constância,86.3,NaN,NaN
3,16H0505,Idanha-a-Nova,69.1,NaN,NaN
4,16J0904,Figueira de Castelo Rodrigo,66.3,NaN,NaN


In [13]:
!wget -O populacao.json https://dados.gov.pt/pt/datasets/r/ce65acd5-0a9d-49f9-92fa-9cfe9f9c2d23 --no-check-certificate

with open('populacao.json') as x: 
    w = json.load(x)
populacao = json_normalize(w[0]['Dados']['2019']) 
populacao.head()

--2020-06-15 19:30:00--  https://dados.gov.pt/pt/datasets/r/ce65acd5-0a9d-49f9-92fa-9cfe9f9c2d23
Resolving dados.gov.pt (dados.gov.pt)... 62.28.186.239
Connecting to dados.gov.pt (dados.gov.pt)|62.28.186.239|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd=0008272&lang=PT [following]
--2020-06-15 19:30:00--  https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd=0008272&lang=PT
Resolving www.ine.pt (www.ine.pt)... 104.22.20.250, 172.67.41.182, 104.22.21.250, ...
Connecting to www.ine.pt (www.ine.pt)|104.22.20.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘populacao.json’

populacao.json          [     <=>            ]   1.31M  1.13MB/s    in 1.2s    

2020-06-15 19:30:02 (1.13 MB/s) - ‘populacao.json’ saved [1378814]



,geocod,geodsg,dim_3,dim_3_t,dim_4,dim_4_t,valor
0,16J0912,Seia,T,HM,T,Total,22234
1,16J0906,Gouveia,T,HM,T,Total,12387
2,16J0905,Fornos de Algodres,T,HM,T,Total,4528
3,16B1012,Óbidos,T,HM,T,Total,11850
4,16B1005,Bombarral,T,HM,T,Total,12558


In [14]:
!wget -O expat.json https://dados.gov.pt/pt/datasets/r/4c29566e-0c10-4a0b-a070-16c0afaa0af4 --no-check-certificate

with open('expat.json') as y: 
    z = json.load(y)
expat = json_normalize(o[0]['Dados']['2018']) 
expat.head()

--2020-06-15 19:30:36--  https://dados.gov.pt/pt/datasets/r/4c29566e-0c10-4a0b-a070-16c0afaa0af4
Resolving dados.gov.pt (dados.gov.pt)... 62.28.186.239
Connecting to dados.gov.pt (dados.gov.pt)|62.28.186.239|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd=0001274&lang=PT [following]
--2020-06-15 19:30:37--  https://www.ine.pt/ine/json_indicador/pindica.jsp?op=2&varcd=0001274&lang=PT
Resolving www.ine.pt (www.ine.pt)... 104.22.20.250, 172.67.41.182, 104.22.21.250, ...
Connecting to www.ine.pt (www.ine.pt)|104.22.20.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847 [application/json]
Saving to: ‘expat.json’

expat.json          100%[===================>]     847  --.-KB/s    in 0.002s  

2020-06-15 19:30:38 (501 KB/s) - ‘expat.json’ saved [847/847]



,geocod,geodsg,valor,sinal_conv,sinal_conv_desc
0,16H0506,Oleiros,51,NaN,NaN
1,16J0503,Covilhã,50,NaN,NaN
2,16J0501,Belmonte,41,NaN,NaN
3,16B1108,Lourinhã,109,NaN,NaN
4,1701107,Loures,50,NaN,NaN
